In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import torchsummary

## **Data**
http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-iamges-idx3-ubyte.gz  
http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz  
http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-iamges-idx3-ubyte.gz  
http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-iamges-idx1-ubyte.gz  
`{root}\FashionMNIST\raw`

In [2]:
trans = transforms.Compose([transforms.Resize((32, 32)),  # upscale
                            transforms.ToTensor()])

data_train = torchvision.datasets.FashionMNIST(
    root='./data', train=True, transform=trans, download=False 
)
data_val = torchvision.datasets.FashionMNIST(
    root='./data', train=False, transform=trans, download=False
)

In [3]:
data_train

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )

In [4]:
data_val

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )

In [5]:
image, label = data_train[0]  # [image, label]
print(image.shape) # (channel, height, weight)
print(label)

torch.Size([1, 32, 32])
9


## **LeNet**

In [6]:
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), padding=2), nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=(2, 2), stride=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), padding=2), nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=(2, 2), stride=2),
            nn.Flatten(),
            nn.Linear(in_features=1024, out_features=120), nn.Sigmoid(),
            nn.Linear(in_features=120, out_features=84), nn.Sigmoid(),
            nn.Linear(in_features=84, out_features=num_classes)
        )

    def forward(self, X):  # X.shape =(batch_size, channel, height, width)
        return self.net(X)

    def _myinit(self, module):
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)

    def myinit(self):
        self.apply(self._myinit)  # apply는 submodule을 재귀적으로 순회

In [7]:
model = LeNet()
model.myinit()

In [8]:
torchsummary.summary(model, input_size=(1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 32, 32]             156
           Sigmoid-2            [-1, 6, 32, 32]               0
         AvgPool2d-3            [-1, 6, 16, 16]               0
            Conv2d-4           [-1, 16, 16, 16]           2,416
           Sigmoid-5           [-1, 16, 16, 16]               0
         AvgPool2d-6             [-1, 16, 8, 8]               0
           Flatten-7                 [-1, 1024]               0
            Linear-8                  [-1, 120]         123,000
           Sigmoid-9                  [-1, 120]               0
           Linear-10                   [-1, 84]          10,164
          Sigmoid-11                   [-1, 84]               0
           Linear-12                   [-1, 10]             850
Total params: 136,586
Trainable params: 136,586
Non-trainable params: 0
-------------------------------

## **Training**

In [9]:
batch_size = 128

train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(data_val, batch_size=batch_size, shuffle=False)

In [10]:
model = LeNet()
model.myinit()

In [11]:
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

In [12]:
def accuracy(y_hat, y):
    # y_hat: (B, q)
    # y: (B)
    preds = y_hat.argmax(axis=1).type(y.dtype)  # (B)
    compare = (preds == y).type(torch.float32)  # (B)
    return compare.sum()

In [13]:
%%time
for i in range(10):
    model.train()

    train_loss = 0
    num_train_batches = 0
    for X, y in train_loader:
        optimizer.zero_grad()
        y_hat = model(X)
        loss = F.cross_entropy(y_hat, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        num_train_batches += 1

    model.eval()
    with torch.no_grad():
        val_loss = 0
        num_val_batches = 0
        val_acc = 0
        total = 0
        for X, y in val_loader:
            y_hat = model(X)
            loss = F.cross_entropy(y_hat, y)
            val_loss += loss.item()
            num_val_batches += 1
            val_acc += accuracy(y_hat, y)
            total += y.numel()
        
    print(f'epoch={i} | train_loss={train_loss/num_train_batches:.4f} | val_loss={val_loss/num_val_batches:.4f} | val_acc={val_acc/total:.4f}')

epoch=0 | train_loss=2.3085 | val_loss=2.3074 | val_acc=0.1000
epoch=1 | train_loss=2.3051 | val_loss=2.3006 | val_acc=0.1000
epoch=2 | train_loss=2.1411 | val_loss=1.5327 | val_acc=0.5355
epoch=3 | train_loss=1.2074 | val_loss=1.0390 | val_acc=0.6074
epoch=4 | train_loss=0.9507 | val_loss=0.8705 | val_acc=0.6780
epoch=5 | train_loss=0.8116 | val_loss=0.7860 | val_acc=0.6953
epoch=6 | train_loss=0.7274 | val_loss=0.7085 | val_acc=0.7303
epoch=7 | train_loss=0.6781 | val_loss=0.6940 | val_acc=0.7247
epoch=8 | train_loss=0.6464 | val_loss=0.6641 | val_acc=0.7441
epoch=9 | train_loss=0.6228 | val_loss=0.6250 | val_acc=0.7668
CPU times: total: 26min 37s
Wall time: 4min 29s
